In [ ]:
!pip install sovai[full]

## Clustering Module

There are two types of clustering models, time-series, and cross-sectional. CS involves feature extraction and clustering algorithm. 

In [38]:
import sovai as sov

sov.token_auth(token="visit https://sov.ai/profile for your token")

#### Processed Dataset

In [43]:
# Load ratios - takes around 5 mins to load data 
df_accounting = sov.data("accounting/weekly", start_date="2023-01-26", purge_cache=True)

Loading partitions: 100%|██████████| 839/839 [00:13<00:00, 60.78partition/s]


In [44]:
df_accounting.shape

(657188, 81)

In [45]:
df_accounting.head()

cash_short_term  operating_working_capital  \
ticker date                                                     
A      2023-01-27   1151500032.000             1015000000.000   
       2023-02-03   1171200000.000             1038000000.000   
       2023-02-10   1190899968.000             1061000000.000   
       2023-02-17   1210599936.000             1084000000.000   
       2023-02-24   1230300032.000             1107000064.000   

                   total_nonoperating_assets  total_operating_assets  \
ticker date                                                            
A      2023-01-27             5968999936.000          8745499648.000   
       2023-02-03             5977600000.000          8764399616.000   
       2023-02-10             5986200064.000          8783299584.000   
       2023-02-17             5994800128.000          8802199552.000   
       2023-02-24             6003400192.000          8821099520.000   

                   accounts_receivable  cash_equiv_usd  cash_equivalents  \
ticker date                                                                
A      2023-01-27       1432000000.000  1151500032.000    1151500032.000   
       2023-02-03       1437400064.000  1171200000.000    1171200000.000   
       2023-02-10       1442800000.000  1190899968.000    1190899968.000   
       2023-02-17       1448199936.000  1210599936.000    1210599936.000   
       2023-02-24       1453600000.000  1230300032.000    1230300032.000   

                   current_assets  current_investments  intangible_assets  \
ticker date                                                                 
A      2023-01-27  3928000000.000                0.000     4783000064.000   
       2023-02-03  3958000128.000                0.000     4784999936.000   
       2023-02-10  3988000000.000                0.000     4786999808.000   
       2023-02-17  4017999872.000                0.000     4789000192.000   
       2023-02-24  4048000000.000                0.000     4791000064.000   

                   inventory_amount  non_current_assets  \
ticker date                                               
A      2023-01-27    1074499968.000      6797499904.000   
       2023-02-03    1081799936.000      6806199808.000   
       2023-02-10    1089100032.000      6814900224.000   
       2023-02-17    1096400000.000      6823600128.000   
       2023-02-24    1103699968.000      6832300032.000   

                   non_current_investments  property_plant_equipment_net  \
ticker date                                                                
A      2023-01-27            191500000.000                1123500032.000   
       2023-02-03            190800000.000                1128199936.000   
       2023-02-10            190100000.000                1132899968.000   
       2023-02-17            189400000.000                1137600000.000   
       2023-02-24            188700000.000                1142300032.000   

                   tax_assets    total_assets  total_investments  \
ticker date                                                        
A      2023-01-27       0.000 10725499904.000      191500000.000   
       2023-02-03       0.000 10764199936.000      190800000.000   
       2023-02-10       0.000 10802899968.000      190100000.000   
       2023-02-17       0.000 10841600000.000      189400000.000   
       2023-02-24       0.000 10880300032.000      188700000.000   

                   accum_other_comp_income  adjusted_parent_equity  \
ticker date                                                          
A      2023-01-27           -313500000.000          5456999936.000   
       2023-02-03           -306800000.000          5487399936.000   
       2023-02-10           -300100000.000          5517799936.000   
       2023-02-17           -293400000.000          5548199936.000   
       2023-02-24           -286700000.000          5578599936.000   

                   book_equity_value     equity_usd  retained_earnings

In [46]:
# Choose long enough history for the model to train
df_mega = df_accounting.select_stocks("mega").date_range("2023-03-02")

### Panel Clustering

This clustering methodology takes multivariate panel datasets and represents them according to the centroids that capture the main patterns within the time series data. 

#### All Features (5 minutes)
First think we can do is to calculate the clusters according to all the features as compared accross all the tickers in the dataset. 

In [47]:
df_mega.head()

cash_short_term  operating_working_capital  \
ticker date                                                     
AAPL   2023-03-03  52744847360.000           -49020690432.000   
       2023-03-10  53092306944.000           -49180614656.000   
       2023-03-17  53439770624.000           -49340538880.000   
       2023-03-24  53787230208.000           -49500463104.000   
       2023-03-31  54134693888.000           -49660383232.000   

                   total_nonoperating_assets  total_operating_assets  \
ticker date                                                            
AAPL   2023-03-03            21414768640.000         92565692416.000   
       2023-03-10            21781460992.000         91364614144.000   
       2023-03-17            22148153344.000         90163535872.000   
       2023-03-24            22514845696.000         88962457600.000   
       2023-03-31            22881538048.000         87761387520.000   

                   accounts_receivable  cash_equiv_usd  cash_equivalents  \
ticker date                                                                
AAPL   2023-03-03      48555077632.000 21812539392.000   21812539392.000   
       2023-03-10      47148847104.000 22131922944.000   22131922944.000   
       2023-03-17      45742616576.000 22451308544.000   22451308544.000   
       2023-03-24      44336386048.000 22770692096.000   22770692096.000   
       2023-03-31      42930155520.000 23090077696.000   23090077696.000   

                    current_assets  current_investments  intangible_assets  \
ticker date                                                                  
AAPL   2023-03-03 123895767040.000      30932307968.000              0.000   
       2023-03-10 122675462144.000      30960384000.000              0.000   
       2023-03-17 121455157248.000      30988462080.000              0.000   
       2023-03-24 120234844160.000      31016538112.000              0.000   
       2023-03-31 119014539264.000      31044616192.000              0.000   

                   inventory_amount  non_current_assets  \
ticker date                                               
AAPL   2023-03-03    7023692288.000    218362920960.000   
       2023-03-10    7074615296.000    218461159424.000   
       2023-03-17    7125538304.000    218559381504.000   
       2023-03-24    7176461312.000    218657619968.000   
       2023-03-31    7227384832.000    218755842048.000   

                   non_current_investments  property_plant_equipment_net  \
ticker date                                                                
AAPL   2023-03-03         112976846848.000               43088539648.000   
       2023-03-10         112697311232.000               43122921472.000   
       2023-03-17         112417767424.000               43157307392.000   
       2023-03-24         112138231808.000               43191693312.000   
       2023-03-31         111858696192.000               43226075136.000   

                   tax_assets     total_assets  total_investments  \
ticker date                                                         
AAPL   2023-03-03       0.000 342258679808.000   143909158912.000   
       2023-03-10       0.000 341136605184.000   143657697280.000   
       2023-03-17       0.000 340014530560.000   143406235648.000   
       2023-03-24       0.000 338892455936.000   143154774016.000   
       2023-03-31       0.000 337770381312.000   142903312384.000   

                   accum_other_comp_income  adjusted_parent_equity  \
ticker date                                                          
AAPL   2023-03-03         -12553230336.000         58398076928.000   
       2023-03-10         -12463538176.000         58815844352.000   
       2023-03-17         -12373846016.000         59233615872.000   
       2023-03-24         -12284153856.000         59651383296.000   
       2023-03-31         -12194461696.000         60069154816.000   

                   book_equity_value      equity_usd  re

In [48]:
df_cluster = df_mega.cluster()

We will remove 0 tickers for which sufficient data does not exist, they are probably delisted


In [49]:
df_cluster.tail()

Centroid 0  Centroid 1  Centroid 2  Centroid 3  Centroid 4  \
ticker date                                                                     
XOM    2025-04-11       0.014       0.130       0.239       0.301       0.221   
       2025-04-18       0.004       0.128       0.243       0.312       0.231   
       2025-04-25      -0.005       0.126       0.247       0.320       0.242   
       2025-05-02      -0.014       0.124       0.250       0.326       0.251   
       2025-05-09      -0.021       0.122       0.254       0.327       0.261   

                   Centroid 5  Centroid 6  Centroid 7  Centroid 8      labels  
ticker date                                                                    
XOM    2025-04-11       0.131       0.187       0.129       0.149  Centroid 3  
       2025-04-18       0.134       0.186       0.126       0.152  Centroid 3  
       2025-04-25       0.137       0.184       0.124       0.155  Centroid 3  
       2025-05-02       0.139       0.182       0.121       0.159  Centroid 3  
       2025-05-09       0.141       0.179       0.118       0.164  Centroid 3

#### Specific Features
We can also focus on any specific feature in the dataset of tens of features like `total_debt`, `total_assets`, or as we are using below `ebit`. 

In [50]:
df_cluster_ebit = df_mega.cluster(features=["ebit"]); df_cluster_ebit.tail()

We will remove 0 tickers for which sufficient data does not exist, they are probably delisted


Centroid 0  Centroid 1  Centroid 2  Centroid 3  Centroid 4  \
ticker date                                                                     
XOM    2025-04-11      -0.645      -0.317       0.013      -0.479      -0.555   
       2025-04-18      -0.653      -0.324      -0.011      -0.501      -0.603   
       2025-04-25      -0.656      -0.326      -0.037      -0.523      -0.654   
       2025-05-02      -0.655      -0.321      -0.065      -0.547      -0.709   
       2025-05-09      -0.650      -0.311      -0.094      -0.571      -0.730   

                   Centroid 5  Centroid 6  Centroid 7  Centroid 8      labels  
ticker date                                                                    
XOM    2025-04-11      -0.474      -0.691       0.712       0.153  Centroid 7  
       2025-04-18      -0.488      -0.717       0.746       0.179  Centroid 7  
       2025-04-25      -0.501      -0.741       0.775       0.204  Centroid 7  
       2025-05-02      -0.513      -0.765       0.799       0.227  Centroid 7  
       2025-05-09      -0.525      -0.788       0.817       0.248  Centroid 7

We can also specify our own selection of multiple features like ``features=["total_assets","total_debt","ebit"]``.


In [51]:
df_mega.cluster(features=["total_assets","total_debt","ebit"])

We will remove 0 tickers for which sufficient data does not exist, they are probably delisted


Centroid 0  Centroid 1  Centroid 2  Centroid 3  Centroid 4  \
ticker date                                                                     
AAPL   2023-03-03       0.001      -0.003       0.001       0.007       0.001   
       2023-03-10       0.001      -0.007       0.002       0.015       0.001   
       2023-03-17       0.001      -0.012       0.001       0.022       0.001   
       2023-03-24       0.001      -0.017      -0.001       0.029       0.000   
       2023-03-31       0.001      -0.024      -0.003       0.033      -0.001   
...                       ...         ...         ...         ...         ...   
XOM    2025-04-11       0.583      -0.140      -0.130       0.345       0.178   
       2025-04-18       0.587      -0.149      -0.138       0.357       0.195   
       2025-04-25       0.590      -0.157      -0.143       0.369       0.216   
       2025-05-02       0.593      -0.164      -0.145       0.378       0.240   
       2025-05-09       0.595      -0.170      -0.144       0.383       0.268   

                   Centroid 5  Centroid 6  Centroid 7  Centroid 8      labels  
ticker date                                                                    
AAPL   2023-03-03       0.005       0.005       0.007      -0.003  Centroid 8  
       2023-03-10       0.010       0.009       0.014      -0.006  Centroid 8  
       2023-03-17       0.015       0.012       0.020      -0.009  Centroid 8  
       2023-03-24       0.019       0.014       0.026      -0.011  Centroid 8  
       2023-03-31       0.023       0.015       0.030      -0.015  Centroid 8  
...                       ...         ...         ...         ...         ...  
XOM    2025-04-11       0.270       0.394       0.328      -0.020  Centroid 0  
       2025-04-18       0.259       0.392       0.303      -0.009  Centroid 0  
       2025-04-25       0.249       0.389       0.278      -0.000  Centroid 0  
       2025-05-02       0.242       0.386       0.255       0.008  Centroid 0  
       2025-05-09       0.240       0.399       0.233       0.015  Centroid 0  

[8510 rows x 10 columns]

#### Downstream Calculations
There are many things you can do once the data is clustered in time series, for one, you can take the standard deviation of the standard deviation of similarity accross clusters. 

In [52]:
import pandas as pd

def transform_df(merged_df):
    # Reset the index to have 'ticker' and 'date' as columns
    df_reset = merged_df.reset_index()

    # Identify the numerical columns (assuming they start with 'Centroid')
    centroid_columns = [col for col in df_reset.columns if col.startswith('Centroid') and col != 'Centroid labels']

    # Calculate the average of the centroid columns
    df_reset['average'] = df_reset[centroid_columns].std(axis=1)

    # Pivot the table to have dates as index and tickers as columns
    transformed_df = df_reset.pivot(index='date', columns='ticker', values='average')

    return transformed_df

# Use the function
transformed_df = transform_df(df_cluster)
max_date = transformed_df.index.max()
sorted_df = (transformed_df.query("date == @max_date")
                           .T
                           .reset_index()
                           .sort_values(by=max_date, ascending=False)
                           .reset_index(drop=True))


Companies who are stable and stay within their accounting cluster over time. 

In [53]:
transformed_df.std().sort_values(ascending=False).tail(10)

ticker
SAP     0.019
WFC     0.019
JNJ     0.019
CMCSA   0.019
XOM     0.016
MCD     0.016
HD      0.015
KO      0.015
BRK.B   0.013
IBM     0.012
dtype: float64

Companies who are see-sawing through accounting clusters over time. 

In [54]:
transformed_df.std().sort_values(ascending=False).head(10)

ticker
NVDA   0.052
LLY    0.045
CSCO   0.044
PLTR   0.042
CRM    0.041
PDD    0.040
META   0.040
TSM    0.037
AMZN   0.036
QCOM   0.035
dtype: float64

#### Distance Cluster

We can use our in-built distance functionality to get the distances between the ticker-cluster combinations.

In [55]:
df_dist = df_cluster.drop(columns=["labels"]).distance(orient="time-series"); df_dist

,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMZN,ASML,AVGO,AXP,AZN,BA,BABA,BAC,BHP,BP,BRK.B,BUD,C,CAT,CMCSA,COST,CRM,CSCO,CVX,DHR,DIS,GE,GOOGL,HD,IBM,INTC,ISRG,JNJ,JPM,KO,LIN,LLY,MA,MCD,META,MRK,MS,MSFT,NFLX,NKE,NOW,NVDA,NVO,NVS,ORCL,PDD,PEP,PFE,PG,PLTR,PM,PYPL,QCOM,SAP,T,TM,TMO,TMUS,TSLA,TSM,TTE,UNH,V,VZ,WFC,WMT,XOM
AAPL,0.000,0.507,0.487,0.676,0.536,0.596,0.403,0.374,0.456,0.485,0.494,0.518,0.687,0.301,0.623,0.644,0.642,0.699,0.638,0.697,0.473,0.555,0.563,0.373,0.806,0.790,0.519,0.464,0.648,0.422,0.674,0.241,0.584,0.492,0.837,0.574,0.801,0.464,0.598,0.429,0.338,0.339,0.640,0.611,0.516,0.410,0.642,0.410,0.431,0.419,0.738,0.592,0.369,0.468,0.804,0.512,0.405,0.654,0.304,0.365,0.540,0.701,0.438,0.291,0.648,0.608,0.477,0.635,0.676,0.455,0.682,0.497,0.462,0.642
ABBV,0.507,0.000,0.340,0.296,0.122,0.360,0.248,0.257,0.256,0.233,0.246,0.251,0.583,0.246,0.222,0.528,0.493,0.351,0.505,0.424,0.303,0.187,0.216,0.551,0.448,0.580,0.441,0.219,0.278,0.246,0.213,0.469,0.435,0.187,0.778,0.201,0.205,0.264,0.338,0.122,0.168,0.141,0.292,0.291,0.297,0.214,0.392,0.302,0.210,0.247,0.838,0.316,0.239,0.150,0.570,0.185,0.236,0.256,0.422,0.201,0.678,0.609,0.281,0.354,0.519,0.218,0.307,0.551,0.288,0.167,0.372,0.142,0.290,0.342
ABT,0.487,0.340,0.000,0.245,0.230,0.314,0.115,0.124,0.154,0.300,0.304,0.241,0.490,0.224,0.190,0.645,0.457,0.364,0.703,0.440,0.374,0.241,0.248,0.442,0.485,0.497,0.454,0.224,0.309,0.209,0.290,0.403,0.574,0.202,0.845,0.252,0.337,0.250,0.245,0.254,0.197,0.222,0.307,0.267,0.285,0.185,0.604,0.250,0.173,0.276,0.771,0.362,0.272,0.330,0.710,0.265,0.113,0.433,0.318,0.198,0.406,0.648,0.406,0.287,0.565,0.305,0.351,0.652,0.246,0.290,0.348,0.286,0.302,0.267
ACN,0.676,0.296,0.245,0.000,0.133,0.218,0.168,0.230,0.183,0.150,0.238,0.141,0.607,0.416,0.121,0.396,0.239,0.174,0.581,0.305,0.440,0.248,0.234,0.487,0.322,0.556,0.432,0.180,0.108,0.132,0.110,0.506,0.527,0.072,0.885,0.111,0.154,0.300,0.106,0.245,0.264,0.212,0.222,0.190,0.122,0.152,0.395,0.202,0.112,0.145,0.733,0.186,0.272,0.270,0.629,0.259,0.155,0.247,0.420,0.180,0.512,0.480,0.255,0.479,0.455,0.148,0.298,0.552,0.201,0.123,0.369,0.340,0.195,0.202
ADBE,0.536,0.122,0.230,0.133,0.000,0.271,0.129,0.132,0.106,0.108,0.134,0.120,0.652,0.230,0.101,0.402,0.345,0.203,0.513,0.381,0.330,0.205,0.132,0.435,0.394,0.502,0.324,0.161,0.138,0.112,0.146,0.401,0.410,0.057,0.889,0.087,0.140,0.203,0.147,0.119,0.113,0.106,0.218,0.268,0.114,0.166,0.449,0.162,0.089,0.129,0.775,0.221,0.147,0.095,0.589,0.148,0.123,0.210,0.301,0.114,0.520,0.508,0.207,0.290,0.409,0.092,0.265,0.514,0.205,0.084,0.324,0.111,0.145,0.204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V,0.455,0.167,0.290,0.123,0.084,0.267,0.123,0.193,0.149,0.091,0.165,0.132,0.595,0.307,0.186,0.478,0.248,0.197,0.466,0.448,0.252,0.186,0.135,0.399,0.474,0.536,0.277,0.121,0.121,0.057,0.189,0.347,0.326,0.029,0.794,0.087,0.187,0.234,0.154,0.095,0.111,0.060,0.201,0.234,0.069,0.094,0.440,0.127,0.067,0.051,0.741,0.174,0.116,0.114,0.692,0.122,0.119,0.159,0.286,0.102,0.498,0.568,0.106,0.336,0.439,0.117,0.251,0.525,0.232,0.000,0.443,0.201,0.129,0.263
VZ,0.682,0.372,0.348,0.369,0.324,0.694,0.424,0.354,0.321,0.486,0.478,0.477,0.649,0.338,0.297,0.617,0.652,0.455,0.833,0.304,0.367,0.240,0.325,0.767,0.489,0.571,0.628,0.347,0.520,0.320,0.466,0.756,0.649,0.375,0.830,0.379,0.341,0.423,0.356,0.329,0.423,0.410,0.387,0.406,0.409,0.370,0.366,0.463,0.421,0.437,0.821,0.464,0.528,0.339,0.517,0.263,0.430,0.536,0.558,0.367,0.659,0.856,0.622,0.535,0.547,0.409,0.489,0.757,0.463,0.443,0.000,0.381,0.330,0.452
WFC,0.497,0.142,0.286,0.340,0.111,0.423,0.184,0.170,0.140,0.235,0.168,0.209,0.629,0.203,0.209,0.441,0.439,0.293,0.482,0.408,0.244,0.288,0.179,0.516,0.556,0.529,0.263,0.311,0.300,0.231,0.314,0.456,0.361,0.191,0.780,0.182,0.265,0.214,0.286,0.172,0.078,0.162,0

Distance calculation for companies with similar clusters:

In [56]:
df_dist.sort_values(["AMZN"])[["AMZN"]].T

,AMZN,PLTR,AMD,ASML,NVDA,NOW,LIN,MCD,AXP,PDD,ABT,QCOM,ADBE,PYPL,TMO,WMT,MSFT,ISRG,MA,LLY,GOOGL,BAC,AZN,META,JPM,AVGO,WFC,NFLX,TSM,NVO,MS,V,PEP,PG,CMCSA,TSLA,BABA,ACN,DHR,DIS,TM,BRK.B,ABBV,COST,ORCL,PM,UNH,MRK,XOM,GE,KO,CAT,CRM,AIG,IBM,HD,VZ,AAPL,C,SAP,BP,INTC,NKE,CVX,BHP,BUD,TMUS,CSCO,BA,PFE,T,NVS,TTE,JNJ
AMZN,0.000,0.063,0.075,0.077,0.088,0.105,0.106,0.106,0.108,0.123,0.124,0.127,0.132,0.132,0.134,0.135,0.139,0.142,0.144,0.146,0.146,0.158,0.159,0.159,0.163,0.163,0.170,0.176,0.180,0.188,0.190,0.193,0.199,0.208,0.210,0.217,0.219,0.230,0.232,0.238,0.250,0.252,0.257,0.258,0.261,0.263,0.280,0.283,0.284,0.285,0.295,0.300,0.302,0.321,0.324,0.343,0.354,0.374,0.402,0.404,0.415,0.424,0.499,0.503,0.518,0.591,0.593,0.605,0.608,0.647,0.675,0.700,0.706,0.859


What about ebit clustering distance?

In [57]:
df_cluster_ebit.drop(columns=["labels"]).distance(orient="time-series").sort_values(["AMZN"])[["AMZN"]].T

,AMZN,MSFT,META,NVO,AMD,NVDA,V,CRM,NOW,BP,DHR,ISRG,LIN,GOOGL,PLTR,BHP,TMO,PDD,CVX,MA,TM,XOM,AXP,ABT,TMUS,PYPL,NFLX,COST,TSLA,QCOM,BA,ACN,GE,ORCL,BUD,CSCO,CMCSA,BAC,TTE,PG,MRK,CAT,LLY,NVS,ADBE,MS,PFE,MCD,AAPL,TSM,JPM,DIS,T,AIG,C,UNH,INTC,KO,AZN,PM,IBM,ABBV,VZ,BRK.B,HD,SAP,WFC,ASML,NKE,PEP,AVGO,JNJ,BABA,WMT
AMZN,0.000,0.006,0.009,0.023,0.026,0.026,0.033,0.037,0.037,0.048,0.051,0.054,0.055,0.057,0.062,0.076,0.076,0.077,0.105,0.108,0.109,0.120,0.135,0.141,0.161,0.172,0.179,0.203,0.205,0.245,0.263,0.281,0.294,0.302,0.307,0.331,0.334,0.338,0.356,0.359,0.361,0.363,0.369,0.376,0.377,0.429,0.449,0.461,0.466,0.466,0.473,0.492,0.507,0.551,0.551,0.579,0.581,0.587,0.602,0.602,0.613,0.620,0.622,0.700,0.726,0.742,0.749,0.754,0.782,0.792,0.803,0.844,0.846,0.897


#### Summary
This gives you a quick summary of the last 6-months data

In [58]:
df_mega.cluster("summary")

We will remove 0 tickers for which sufficient data does not exist, they are probably delisted


,Cluster,Divergence,Growing Centroid,Falling Centroid,Average Standard Deviation,Average Correlation
ticker,,,,,,
AAPL,Centroid 8,20.688,Centroid 5 (+23.09%),Centroid 1 (+4.79%),0.065,0.175
ABBV,Centroid 2,10.008,Centroid 3 (+11.24%),Centroid 0 (-3.78%),0.062,0.134
ABT,Centroid 6,23.469,Centroid 6 (+23.47%),Centroid 2 (+2.13%),0.082,0.229
ACN,Centroid 1,10.998,Centroid 1 (+11.00%),Centroid 2 (-2.95%),0.103,0.206
ADBE,Centroid 6,13.683,Centroid 6 (+13.68%),Centroid 2 (-7.18%),0.107,0.244
...,...,...,...,...,...,...
V,Centroid 8,17.189,Centroid 8 (+17.19%),Centroid 3 (+2.26%),0.121,0.262
VZ,Centroid 8,18.220,Centroid 8 (+18.22%),Centroid 2 (-7.08%),0.061,0.128
WFC,Centroid 6,11.367,Centroid 8 (+13.06%),Centroid 7 (-4.24%),0.080,0.171


#### Vizualisation
Each colored line represents a distance to centroid of the cluster. The centroid is the average pattern of all time series assigned to that cluster. These are similarity scores (based on cross-correlation). Selecting features shows you the different shapes over time. 

In [60]:
df_mega.cluster("line_plot")

We will remove 0 tickers for which sufficient data does not exist, they are probably delisted


In [61]:
df_mega.cluster("scatter_plot")

In [62]:
df_mega.cluster("animation_plot")

We will remove 0 tickers for which sufficient data does not exist, they are probably delisted
